In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../EEG_Data.csv")

In [3]:
X = data.iloc[:, :4]
Y = data.iloc[:, 4:]

In [4]:
X_train = []
for x in range(0, len(X["eeg1"])):
    X_train.append([X["eeg1"][x], X["eeg3"][x], X["eeg4"][x]])
X_train = np.array(X_train, dtype='float')

X_train

array([[ 0.32829469, -0.1665177 ,  0.56468612],
       [-0.85307294, -0.80708277, -0.10424572],
       [-0.0170797 , -0.7498735 ,  0.09149241],
       ...,
       [-0.2469368 , -0.5131385 ,  0.25170445],
       [-0.09700678, -0.63461101, -0.01072684],
       [ 0.36853725, -0.3391129 ,  0.342383  ]])

In [5]:
Y_train = []
for y in range(0, len(Y["focused"])):
    Y_train.append([Y["focused"][y]])
Y_train = np.array(Y_train, dtype='float')

Y_train

array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [0.]])

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class EEG_Model(nn.Module):
    def __init__(self):
        super(EEG_Model, self).__init__()
        self.fc1 = nn.Linear(3, 15)
        self.fc2 = nn.Linear(15, 15)
        self.fc3 = nn.Linear(15, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        # add layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.sigmoid(self.fc3(x))        
        return x

# initialize the NN
model = EEG_Model()
model = model.double()

print(model)

EEG_Model(
  (fc1): Linear(in_features=3, out_features=15, bias=True)
  (fc2): Linear(in_features=15, out_features=15, bias=True)
  (fc3): Linear(in_features=15, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [8]:
# specify loss function
criterion = nn.MSELoss()

# specify loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
n_epochs = 100

for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for i in range(len(X_train)):
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        # forward pass: compute predicted outputs by passing inputs to the model
        x = X_train[i]
        x = torch.from_numpy(x)
        
        y = Y_train[i]
        y = torch.from_numpy(y)

        outputs = model(x)
        
        # calculate the loss
        loss = criterion(outputs, y)
        
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        
        # perform a single optimization step (parameter update)
        optimizer.step()
        
        # update running training loss
        train_loss += loss.item()
            
    # print avg training statistics 
    train_loss = train_loss/len(X_train)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, 
        train_loss
        ))

//anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1 	Training Loss: 0.149250
Epoch: 2 	Training Loss: 0.141385
Epoch: 3 	Training Loss: 0.138883
Epoch: 4 	Training Loss: 0.135969
Epoch: 5 	Training Loss: 0.135684
Epoch: 6 	Training Loss: 0.134967
Epoch: 7 	Training Loss: 0.135143
Epoch: 8 	Training Loss: 0.134957
Epoch: 9 	Training Loss: 0.134226
Epoch: 10 	Training Loss: 0.133545
Epoch: 11 	Training Loss: 0.134355
Epoch: 12 	Training Loss: 0.133614
Epoch: 13 	Training Loss: 0.134129
Epoch: 14 	Training Loss: 0.132967
Epoch: 15 	Training Loss: 0.133210
Epoch: 16 	Training Loss: 0.133285
Epoch: 17 	Training Loss: 0.133083
Epoch: 18 	Training Loss: 0.133789
Epoch: 19 	Training Loss: 0.132950
Epoch: 20 	Training Loss: 0.132553
Epoch: 21 	Training Loss: 0.134432
Epoch: 22 	Training Loss: 0.133546
Epoch: 23 	Training Loss: 0.133634
Epoch: 24 	Training Loss: 0.132976
Epoch: 25 	Training Loss: 0.133070
Epoch: 26 	Training Loss: 0.132522
Epoch: 27 	Training Loss: 0.133617
Epoch: 28 	Training Loss: 0.133105
Epoch: 29 	Training Loss: 0.1

In [12]:
torch.save(model.state_dict(), 'EEG_Model.pt')

AttributeError: 'EEG_Model' object has no attribute 'save_state_dict'